In [1]:
import nibabel as nib
import numpy as np
import os
from os import path
import pandas as pd
from scipy.spatial import distance_matrix
from sklearn.cluster import DBSCAN
import sys
from zipfile import ZipFile

In [2]:
from fsedit_notes_helpers import get_diff_data, summarize_edits, get_bm_edits, get_wm_edits, get_cp_edits, get_bfs_edits

In [3]:
subnum = 'CC0007'
typ = 'wm'
basedir='/Users/zeynepenkavi/Downloads/ConteQC'
outdir='/Users/zeynepenkavi/Downloads/ConteQC'

In [4]:
# Check if original zip exists. If not print error and quit
exists = path.isfile(path.join(basedir, 'sub-'+subnum+'.zip'))
if not exists:
    print("Unedited images not found in directory. Quitting...")
    try:
        sys.exit()
    except SystemExit:
        print("Quit because difference images cannot be calculated without unedited images.")
else:
    # Unzip unedited images of potential interest
    toExtract = ['brainmask.mgz', 'wm.mgz', 'brain.finalsurfs.mgz']
    suffix = path.join('sub-'+subnum, 'mri')
    toExtract = [path.join(suffix, s) for s in toExtract]
    with ZipFile(path.join(basedir, 'sub-'+subnum+'.zip'), 'r') as zipObj:
        # Get a list of all archived file names from the zip
        listOfFileNames = zipObj.namelist()
        # Iterate over the file names
        for fileName in listOfFileNames:
            # Check filename endswith csv
            if fileName in toExtract:
                # Extract a single file from zip
                zipObj.extract(fileName, path.join(basedir, 'sub-'+subnum+'_unedited'))
    print("Extracting unedited files complete.")

Extracting unedited files complete.


In [5]:
editp = path.join(basedir, 'sub-'+subnum)
uneditp = path.join(basedir, 'sub-'+subnum+'_unedited')
typ_dict = {'bm': 'brainmask', 'wm': 'wm', 'bfs': 'brain.finalsurfs', 'cp': 'control.dat'}
vol = typ_dict[typ]

In [6]:
if vol == "brain.finalsurfs":
    edit_img = nib.load(path.join(editp,'mri/%s.manedit.mgz')%(vol))
else:
    edit_img = nib.load(path.join(editp,'mri/%s.mgz')%(vol))

edit_data = edit_img.get_fdata()
unedit_img = nib.load(path.join(uneditp,'sub-%s/mri/%s.mgz')%(subnum, vol))
unedit_data = unedit_img.get_fdata()
diff_data = unedit_data - edit_data

In [14]:
diffp = path.join(basedir, 'sub-'+subnum+'_diff')
if not os.path.exists(diffp):
    os.makedirs(diffp)
diff_img = nib.MGHImage(diff_data.astype(np.int32), edit_img.affine)
nib.save(diff_img, os.path.join(diffp, 'diff_%s.mgz')%(vol))

In [11]:
np.unique(diff_data)

array([ -1.,   0.,  93.,  95.,  96.,  97.,  98., 100., 101., 102., 103.])